# Data Processing


In [14]:
# %pip install datasets
# %pip install psycopg2
import pandas as pd


import psycopg2
import json


In [ ]:
rag_df = pd.read_csv("hf://datasets/Sachinkelenjaguri/Resume_dataset/UpdatedResumeDataSet.csv")


**This makes the dataset smaller by selecting Tech orientated job categories. This will reduce the time to vectorise the data and makes the AI Agent more focused on a specific sector. Ideal for CV and Cover letter reviews / generation.**


In [ ]:
roles = [
    'Data Science', 'Web Designing', 
    'Java Developer', 'Automation Testing',
    'Operations Manager', 'Python Developer',
    'DevOps Engineer', 'Network Security Engineer',
    'Database', 'DotNet Developer',
    'Testing', 'Blockchain'
]
rag_df = rag_df[rag_df['Category'].isin(roles)]
rag_df


In [ ]:
-- Ensure the table exists before vectorizing
DROP TABLE IF EXISTS resumes_1;

CREATE TABLE resumes_1 (
    id SERIAL PRIMARY KEY,
    category TEXT NOT NULL,
    content TEXT NOT NULL,
    metadata JSONB
);

-- Create the vectorizer using AI functions

SELECT ai.create_vectorizer(   
    'resumes_1'::regclass,
    formatting => ai.formatting_python_template('Job Role: $category\n\nCV:\n$chunk'),
    embedding => ai.embedding_ollama('deepseek-r1:7b', 3584),
    chunking => ai.chunking_recursive_character_text_splitter('content'),
    destination => 'resume_1_embedding'
);

In [ ]:
DB_PARAMS = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": "5432" 
}


# Function to insert data
def insert_data(df):
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(**DB_PARAMS)
        cur = conn.cursor()
        
        # Insert each row into the database
        for _, row in df.iterrows():
            category = row['Category']
            content = row['Resume']
            metadata = json.dumps({"source": "uploaded_resume"})  # Example metadata

            cur.execute("""
                INSERT INTO resumes_1 (category, content, metadata)
                VALUES (%s, %s, %s)
            """, (category, content, metadata))

        # Commit changes
        conn.commit()
        print("Data inserted successfully.")

    except Exception as e:
        print("Error:", e)
    finally:
        cur.close()
        conn.close()

# Call the function to insert data
insert_data(rag_df)


In [ ]:
big_df = pd.read_csv("hf://datasets/opensporks/resumes/Resume/Resume.csv")


In [ ]:
massive_df = pd.read_csv("hf://datasets/InferencePrince555/Resume-Dataset/updated_data_final_cleaned.csv")


### RAG Function

In [ ]:
CREATE OR REPLACE FUNCTION generate_rag_response(query_text TEXT)
RETURNS TEXT AS $$
DECLARE
   context_chunks TEXT;
   response TEXT;
BEGIN
   -- Perform similarity search to find relevant resumes
   SELECT string_agg(category || ': ' || chunk, E'\n') INTO context_chunks
   FROM
   (
       SELECT category, chunk
       FROM resume_1_embedding
       ORDER BY embedding <=> ai.ollama_embed('deepseek-r1:7b', query_text)
       LIMIT 3
   ) AS relevant_posts;

   -- Generate a summary using llama3
   SELECT ai.ollama_chat_complete
   ( 'deepseek-r1:7b'
   , jsonb_build_array
     ( jsonb_build_object('role', 'system', 'content', 'you are a helpful assistant')
     , jsonb_build_object
       ('role', 'user'
       , 'content', query_text || E'\nUse the following context to respond.\n' || context_chunks
       )
     )
   )->'message'->>'content' INTO response;

   RETURN response;
END;
$$ LANGUAGE plpgsql;